In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle
import numpy as np
import pyarrow.parquet as pq
import pandas as pd
pd.options.display.max_rows = 100
from rich import print
import warnings
import datetime
from astropy.time import Time
import os
import astropy.units as u
from astroquery.gaia import Gaia
from astroquery.simbad import Simbad
import astropy.coordinates as coord
import astropy.units as u
import time
from gfcat_utils import parse_lightcurves_csv,parse_exposure_time,read_image
from matplotlib import gridspec
from astropy.visualization import ZScaleInterval
from matplotlib.ticker import FormatStrFormatter, MaxNLocator
from scipy.interpolate import interp1d
import tqdm

In [2]:
def angularSeparation(ra1, dec1, ra2, dec2):
    d2r = np.pi/180.
    ra2deg = 1./d2r
    d1 = dec1*d2r,
    d2 = dec2*d2r
    r1 = ra1*d2r
    r2 = ra2*d2r
    a = np.sin((d2-d1)/2.)**2.+np.cos(d1)*np.cos(d2)*np.sin((r2-r1)/2.)**2.
    r = 2*np.arcsin(np.sqrt(a))
    return r*ra2deg

def counts2flux(cps, band):
    scale = 1.4e-15 if band == 'FUV' else 2.06e-16
    return scale*cps

def counts2mag(cps, band):
    scale = 18.82 if band == 'FUV' else 20.08
    # This threw a warning if the countrate was negative which happens when
    #  the background is brighter than the source. Suppress.
    with np.errstate(invalid='ignore'):
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            mag = -2.5 * np.log10(cps) + scale
    return mag

def mag2counts(mag, band):
    scale = 18.82 if band == 'FUV' else 20.08
    return 10.**(-(mag-scale)/2.5)

def apcorrect1(radius, band):
    if not band in ['NUV', 'FUV']:
        print("Invalid band.")
        return
    aper = np.array([1.5, 2.3, 3.8, 6.0, 9.0, 12.8, 17.3, 30., 60., 90.])/3600.
    if radius > aper[-1]:
        return 0.
    if band == 'FUV':
        dmag = [1.65, 0.96, 0.36, 0.15, 0.1, 0.09, 0.07, 0.06, 0.03, 0.01]
    else:
        dmag = [2.09, 1.33, 0.59, 0.23, 0.13, 0.09, 0.07, 0.04, -0.00, -0.01]
        if radius > aper[-2]:
            return 0.
    if radius < aper[0]:
        return dmag[0]
    ix = np.where((aper-radius) >= 0.)
    x = [aper[ix[0][0]-1], aper[ix[0][0]]]
    y = [dmag[ix[0][0]-1], dmag[ix[0][0]]]
    m, C = np.polyfit(x, y, 1)
    return m*radius+C

In [3]:
# e37477 has two variables, so use that as a tester

In [4]:
tbl = pd.read_csv('gfcat_visit_table_crossmatched.csv',index_col=None)

for i in tqdm.tqdm(np.arange(len(tbl))):
    visit = tbl.iloc[i]
    e = visit['eclipse']
    edir = str(e).zfill(5)
    epath = f"../data/lightcurves/e{edir}"
    ra,dec = visit['ra'],visit['dec']
    obs = {}
    for band in ['NUV','FUV']:
        fn = f"{epath}/e{edir}-{band[0].lower()}d-30s-photom-17_5.csv"
        try:
            expt = parse_exposure_time(f"{epath}/e{edir}-{band[0].lower()}d-30s-exptime.csv")
            lcs = parse_lightcurves_csv(fn)
            # a few eclipse have multiple variables, so we have to pick the right one
            ix = np.argmin(angularSeparation(ra,dec,
                           np.array([lc['ra'] for lc in lcs]),
                           np.array([lc['dec'] for lc in lcs])))
            lc = lcs[ix]
        except FileNotFoundError:
            #print(f'Failed to find{fn}')
            continue
        obs[band] = [lc,expt]

        # Flag certain eclipses / bands with data quality issues determined during manual QA
        if (((band=='NUV') and (e in [3141,3225,4426,21257,21257,23279,23521,26377,30790,30872,
                                      30875,32012,1366,7416,7927,8147,8565,28681,4406,13749,10796,
                                      10868,28681,30696,31950,])) or
            ((band=='FUV') and (e in [1366,3779,4406,12889,12897,3779,8256,17884,19051,23716,2648,13749,
                                      6513,7129,7835,13112,14671,14825,21750]))):
            tbl.loc[i,f'qa_flag_{band}']=1
        
        
        # There appears to be ramp-up phenomenon in FUV in this eclipse range
        lims = (3,None) if ((band=='FUV') and (e>=12889) and (e<=13856)) else (None,None)
        
        # add observation minimum flux
        ix = np.where(lc['cps']==np.nanmin(lc['cps'][np.where(lc['cps'][lims[0]:]!=0)]))[0][0]
        tbl.loc[i,f'cps_min_{band}']=lc['cps'][ix]
        tbl.loc[i,f'cps_min_err_{band}']=lc['cps_err'][ix]
        tbl.loc[i,f'mag_min_{band}']=counts2mag(lc['cps'][ix],band)
        tbl.loc[i,f'mag_min_err_{band}']=np.abs(
            counts2mag(lc['cps_err'][ix]+lc['cps'][ix],band)-counts2mag(lc['cps'][ix],band))
        
        # observation maximum flux
        ix = np.where(lc['cps']==np.nanmax(lc['cps'][np.where(np.isfinite(lc['cps']))]))[0][0]
        tbl.loc[i,f'cps_max_{band}']=lc['cps'][ix]
        tbl.loc[i,f'cps_max_err_{band}']=lc['cps_err'][ix]
        tbl.loc[i,f'mag_max_{band}']=counts2mag(lc['cps'][ix],band)
        tbl.loc[i,f'mag_max_err_{band}']=np.abs(
            counts2mag(lc['cps_err'][ix]+lc['cps'][ix],band)-counts2mag(lc['cps'][ix],band))

        # observation mean flux
        t = np.array(expt['expt_eff'])[lims[0]:][
            np.where(np.isfinite(lc['cps'][lims[0]:]) & (lc['cps'][lims[0]:]!=0))]
        c = lc['cps'][lims[0]:][
            np.where(np.isfinite(lc['cps'][lims[0]:]) & (lc['cps'][lims[0]:]!=0))]
        tbl.loc[i,f'cps_mean_{band}']=(c*t).sum()/t.sum()
        tbl.loc[i,f'cps_mean_err_{band}']=np.sqrt((c*t).sum())/t.sum()
        tbl.loc[i,f'mag_mean_{band}']=counts2mag((c*t).sum()/t.sum(),band)
        tbl.loc[i,f'mag_mean_err_{band}']=np.abs(
            counts2mag(np.sqrt((c*t).sum())/t.sum()+(c*t).sum()/t.sum(),band)-counts2mag((c*t).sum()/t.sum(),band))

        
        # add valid data start time, stop time, duration
        ix = np.where(np.isfinite(lc['cps']))
        tbl.loc[i,f'tstart_{band}'] = np.round(np.min(np.array(expt['t0'])[ix]))
        tbl.loc[i,f'tstop_{band}'] = np.round(np.max(np.array(expt['t0'])[ix]))
        tbl.loc[i,f'expt_raw_{band}'] = (np.round(np.max(np.array(expt['t0'])[ix])) -
                                        np.round(np.min(np.array(expt['t0'])[ix])))

    # if there is data for both bands, add the color information
    if len(obs.keys())==2:
        _,ix,_ = np.intersect1d(obs['NUV'][1]['t0'],obs['FUV'][1]['t0'][lims[0]:],return_indices=True)
        if len(ix)==0:
            print(f"Times misaligned in e{e}")
        else:
            color = (counts2flux(obs['FUV'][0]['cps'],'FUV')[ix][:-1] /
                     counts2flux(obs['NUV'][0]['cps'],'NUV')[ix][:-1])
            color_err = color * np.sqrt(
                (counts2flux(obs['NUV'][0]['cps_err'],'NUV')[ix][:-1] /
                 counts2flux(obs['NUV'][0]['cps'],    'NUV')[ix][:-1])**2 +
                (counts2flux(obs['FUV'][0]['cps_err'],'FUV')[ix][:-1] /
                 counts2flux(obs['FUV'][0]['cps'],    'FUV')[ix][:-1])**2)
            expt = obs['FUV'][1]
            ix = np.where(color==np.nanmin(color[np.where(color!=0)]))
            tbl.loc[i,'ratio_min'] = color[ix]
            tbl.loc[i,'ratio_min_err'] = color_err[ix]
            ix = np.where(color==np.nanmax(color[np.where(np.isfinite(color))]))
            tbl.loc[i,'ratio_max'] = color[ix]
            tbl.loc[i,'ratio_max_err'] = color_err[ix]
            ix = np.where(lc['cps']==np.nanmax(lc['cps'][np.where(np.isfinite(color))]))
            tbl.loc[i,'ratio_at_nuv_max'] = color[ix]
            tbl.loc[i,'ratio_at_nuv_max_err'] = color_err[ix]
#tbl[['eclipse','cps_min_FUV','cps_max_FUV','cps_min_NUV','cps_max_NUV','cps_mean_FUV','cps_mean_NUV','ratio_max']]

 71%|███████████████████████████████████████████████████████████████████████████████████████████                                      | 1662/2353 [00:12<00:04, 159.94it/s]

Times misaligned in e13549

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2353/2353 [00:16<00:00, 139.64it/s]


In [5]:
apers = [9,12.5,17.5,25,35]
for i in tqdm.tqdm(np.arange(len(tbl))):
    visit = tbl.iloc[i]
    e = visit['eclipse']
    edir = str(e).zfill(5)
    epath = f"../data/lightcurves/e{edir}"
    ra,dec = visit['ra'],visit['dec']
    for band in ['NUV','FUV']:
        try:
            expt = parse_exposure_time(f"{epath}/e{edir}-{band[0].lower()}d-30s-exptime.csv")
        except FileNotFoundError: # no data for this band
            continue
        tbl.loc[i,f'expt_eff_{band}'] = sum(expt['expt_eff'])
        for aper in apers:
            aperstr = str(aper).replace('.','_')
            fn = f"{epath}/e{edir}-{band[0].lower()}d-30s-photom-{aperstr}.csv"
            try:
                lcs = parse_lightcurves_csv(fn)
                # a few eclipse have multiple variables, so we have to pick the right one
                ix = np.argmin(angularSeparation(ra,dec,
                               np.array([lc['ra'] for lc in lcs]),
                               np.array([lc['dec'] for lc in lcs])))
                lc = lcs[ix]
            except FileNotFoundError: # no data for this aperture? but data for this band. shouldn't happen.
                print(f"Error finding {fn}")
                continue
            tbl.loc[i,f'counts_{aperstr}'] = np.sum(lc['counts'])
        aper_area = np.pi*17.5**2
        annulus_area = np.pi*35**2-np.pi*25**2
        tbl.loc[i,f'skybg_counts_{band}']=(tbl.iloc[i]['counts_35']-tbl.iloc[i]['counts_25'])*aper_area/annulus_area
        tbl.loc[i,f'skybg_counts_err_{band}']=np.sqrt(tbl.iloc[i]['counts_35']-tbl.iloc[i]['counts_25'])*aper_area/annulus_area
        tbl.loc[i,f'skybg_flux_{band}']=counts2flux(tbl.iloc[i]['counts_35']-tbl.iloc[i]['counts_25'],band)*aper_area/annulus_area
        tbl.loc[i,f'skybg_flux_err_{band}']=counts2flux(np.sqrt(tbl.iloc[i]['counts_35']-tbl.iloc[i]['counts_25']*aper_area/annulus_area),band)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2353/2353 [00:40<00:00, 58.71it/s]


In [6]:
tbl.replace(np.nan, '--', inplace=True)
tbl.sort_values(['ra'],inplace=True)
tbl.to_csv('gfcat_visit_table_lcstats.csv',index=None)
tbl

,eclipse,obj_id,ra,dec,morphology,simbad_n_match,simbad_otype,simbad_main_id,simbad_distance,simbad_match_offset,...,counts_35,skybg_counts_NUV,skybg_counts_err_NUV,skybg_flux_NUV,skybg_flux_err_NUV,expt_eff_FUV,skybg_counts_FUV,skybg_counts_err_FUV,skybg_flux_FUV,skybg_flux_err_FUV
1208,34085,1628034085.0,1.012610,-4.692878,T,1,Variable*,ATO J001.0125-04.6926,2558.8536335721597,1.0912917153506068,...,20735.364755,1927.191086,31.363521,0.0,0.0,--,--,--,--,--
2241,45476,--,1.235809,32.635346,U,1,Star,TYC 2263-1012-1,882.4567596187787,14.29205763855893,...,80089.190392,3714.976835,43.545219,0.0,0.0,--,--,--,--,--
1064,34227,1337334227.0,2.248695,23.643045,T,1,**,ADS 106 AB,--,1.1062231150441744,...,261858.266497,5347.262066,52.243006,0.0,0.0,--,--,--,--,--
2246,45547,--,2.302683,42.962552,U,1,HighPM*,TYC 2789-591-1,176.85967953026068,13.10140700489793,...,29819.801352,2847.027602,38.120471,0.0,0.0,--,--,--,--,--
495,28485,662128485.0,2.427261,-0.563277,T,1,RRLyrae,ATO J002.4268-00.5629,25380.71065989848,1.9210325389059826,...,779.625709,1813.736469,30.426326,0.0,0.0,1463.819437,179.480941,9.571315,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,28561,587528561.0,359.336534,-12.980567,F,1,"**, Eruptive*",LP 704-15,19.831944105648734,2.9998324595644084,...,827.840623,1626.662597,28.814505,0.0,0.0,1604.555159,117.814459,7.754641,0.0,0.0
2196,44572,--,359.441055,-36.489974,U,0,--,--,--,--,...,7819.939681,1351.389199,26.263503,0.0,0.0,--,--,--,--,--
2195,44572,--,359.454776,-36.505266,U,1,RRLyrae,SB 921,6968.641114982579,2.104872374654677,...,43847.217555,1843.869473,30.678033,0.0,0.0,--,--,--,--,--
1156,28525,1509128525.0,359.491560,-3.689567,F,1,EclBin_Candidate,KIC 60019244,321.75032175032175,1.8061054819406248,...,950.543368,1828.926422,30.55347,0.0,0.0,1550.718663,187.60499,9.785536,0.0,0.0


In [7]:
apers = [9,12.5,17.5,25,35]
print("Estimated aperture corrections on a")
for aper in apers:
    print(f"  =>  {aper}\" aperture is {np.round(100*apcorrect1(aper/60/60,'NUV'),1)}% in NUV and {np.round(100*apcorrect1(aper/60/60,'FUV'),1)}% in FUV")

Estimated aperture corrections on a

=>  9" aperture is 13.0% in NUV and 10.0% in FUV

=>  12.5" aperture is 9.3% in NUV and 9.1% in FUV

=>  17.5" aperture is 7.0% in NUV and 7.0% in FUV

=>  25" aperture is 5.2% in NUV and 6.4% in FUV

=>  35" aperture is 3.3% in NUV and 5.5% in FUV